In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from fsl_mrs.utils.plotting import FID2Spec
import fsl_mrs.utils.mrs_io as mrs_io
from fsl_mrs.utils.preproc import nifti_mrs_proc as proc
from fsl_mrs.utils.plotting import plotly_fit
from fsl_mrs.utils import misc
import fsl_mrs.core as core


#part is choice of particpant, ranges from 001 to 008
part = '004'

In [ ]:
#Load in the water suppressed reference, and the basis
from pathlib import Path

# Path to the directory 
NOTEBOOK_DIR = Path().resolve()
print(NOTEBOOK_DIR)


sup = mrs_io.read_FID(rf'{NOTEBOOK_DIR}/Data_for_fitting/sLaser/{part}/final_sup.nii.gz')

basis_nowater = mrs_io.read_basis(rf'{NOTEBOOK_DIR}/basis_sets/slaser_dkd_30.0_h2o')
#water suppressed already so no water required 
basis_nowater.remove_fid_from_basis('H2O')

In [ ]:
#PARTICIPANT 007 ONLY
#Excessive lipid contamination required an additional lipid peak in the model, included as a simple gaussian
#Treat lipid like macromolecules in futher processing

#fid = misc.create_peak(time_axis=np.arange(0, 8192/6000, 1/6000), sigma=35, amp=0.5, cf = 123.22, ppm=8.31)
#basis_nowater.add_fid_to_basis(fid, 'LIP')

In [ ]:
#Fit water suppressed reference

supmrs = sup.mrs(basis=basis_nowater)
supmrs.check_Basis(repair=True)
supmrs.rescaleForFitting(ind_scaling=['Mac'])

res_sup = supmrs.fit(
    metab_groups=supmrs.parse_metab_groups(['Mac']),
    ppmlim=(0.2, 4.2),
    baseline='spline, moderate',
    model='voigt'
)

plotly_fit(supmrs, res_sup, (0.2, 4.2))

In [ ]:
#Remove water peak from unsuppressed data

raw_unsup = np.load(rf'{NOTEBOOK_DIR}/Data_for_fitting/sLaser/{part}/final_unsup.npy')
base= sup.copy()

base[:] = raw_unsup.conj()

#Align to the water peak
base = proc.shift_to_reference(base, 4.65, (4.5, 4.8))
base = proc.phase_correct(base, (4.5, 4.8))

basis = mrs_io.read_basis(rf'{NOTEBOOK_DIR}/basis_sets/slaser_dkd_30.0_h2o')


mrs1 = base.mrs(basis=basis)
mrs1.keep = ['H2O']

mrs1.check_Basis(repair=True)
mrs1.rescaleForFitting()

#Fit with restricted frequency range and no baseline
res_water = mrs1.fit(
    ppmlim= (-10,4.55),
    baseline='off',
    model='voigt',
)

wat_concs = res_water.getConc()[0] / (mrs1.fid_scaling / (mrs1.basis_scaling[0]))


plt.figure()
_ = res_water.plot(mrs1)


In [ ]:
#Extract the water peak and remove it from the raw data

fid_pred = res_water.predictedFID(mrs1, mode='Full', noBaseline=True)
fid_pred  /= mrs1.scaling['FID']

water = base.copy()
water[:] = fid_pred


metabs = proc.subtract(base, water)

#Factor of 2 is needed as subtract is used for spectral editing normally which includes a divsion by 2
metabs[:] *= 2
metabs = proc.shift_to_reference(metabs, 2.01, (1.9, 2.1), figure=False)

In [ ]:
#Placing the raw sup and unsup data into same nifti_mrs object
#This is so they can be aligned to have the same frequency and phase references as well as possible

x = sup.copy()
a = x.copy()
b = x.copy()

# 2) Create a singleton 5th dimension
a = core.nifti_mrs.reshape(a, (1,), d5='DIM_EDIT')
b = core.nifti_mrs.reshape(b, (1,), d5='DIM_EDIT')

# 3) Merge them along the 5th dimension → size 2
x = core.nifti_mrs.merge([a, b], dimension='DIM_EDIT')

x[:,:,:,:,1] = metabs[:]
x[:,:,:,:,0] = sup[:]

#Performing alignment here
x = proc.align(x, 'DIM_EDIT', ppmlim=(0.2, 4.2))

y = metabs.copy()
y[:] = x[:,:,:,:,1]
z = metabs.copy()
z[:] = x[:,:,:,:,0]


#Optionally save the Raw data arrays for future plotting
#y.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_raw/{part}_unsup.nii.gz')
#z.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_raw/{part}_sup.nii.gz')
#x.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_raw/{part}_difference.nii.gz')

plt.figure(figsize=(6,4))
y.plot(ppmlim=(0.2,4.2))
z.plot(ppmlim=(0.2,4.2))
x.plot(ppmlim=(0.2,4.2))

ax = plt.gca()

# Recolor lines
lines = ax.get_lines()

diff_line = lines[2]
xdata = diff_line.get_xdata()
ydata = diff_line.get_ydata()


#Mutiply differenct by 2 due to proc.subtract reasons and offset
ydata_new = -(2 * ydata + 1e-5)

# Update the line
diff_line.set_ydata(ydata_new)

lines[1].set_color('blue')
lines[0].set_color('red')
lines[2].set_color('orange')

lines[2].set_linewidth(1.8)
lines[1].set_linewidth(1.8)
lines[0].set_linewidth(1.8)

lines[1].set_linestyle((0,(1,1)))

ax.legend(
    [lines[0], lines[1], lines[2]],
    ['GIRF-Corrected Non-WS', 'WS', 'Difference'],
    fontsize=10
)

# Remove y ticks and set label
ax.set_yticks([])
ax.axhline(-1e-5, color='gray', linestyle=':', linewidth=0.7)

#ax.text(ax.get_xlim()[0] - 0.1, 0, '0', va='center', ha='right', fontsize=10, color='gray')
ax.set_ylim(-0.000018, 0.00004)
ax.set_ylabel('Signal', fontsize=11)
ax.set_xlabel("Chemical Shift (ppm)", fontsize=11)
ax.tick_params(axis='x', labelsize=8)
ax.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.6)
ax.set_title('Processed Data for Metabolite Fitting', fontsize = 14)
#ax.set_title('HLSVD Removal of Residual Water', fontsize = 14)
ax.grid(False)  # turns all grids off first
ax.xaxis.grid(False, which='major')
ax.xaxis.grid(False, which='minor')
plt.tight_layout()
plt.show()

In [ ]:
#Fit the water-unsuppressed data

metabs[:] = x[:,:,:,:,1]
basis_nowater = mrs_io.read_basis(rf'{NOTEBOOK_DIR}/basis_sets/slaser_dkd_30.0_h2o')
basis_nowater.remove_fid_from_basis('H2O')
#basis_nowater.add_fid_to_basis(fid, 'LIP')
unsup_mrs = metabs.mrs(basis=basis_nowater)


unsup_mrs.check_Basis(repair=True)
unsup_mrs.rescaleForFitting(ind_scaling=['Mac'])

unsup_mrs.fid_scaling = supmrs.fid_scaling

res_unsup = unsup_mrs.fit(
    metab_groups=unsup_mrs.parse_metab_groups([ 'Mac']),
    ppmlim=(0.2, 4.2),
    baseline='spline, moderate',
    model='voigt'
)


plotly_fit(unsup_mrs, res_unsup, (0.2, 4.2))

In [ ]:
#set up dataframe to compare metabolite concetrations of the suppressed and unsuppressed data

import pandas as pd
pd.set_option('display.float_format', '{:.6f}'.format)

#Get the scalings of the two different datasets correct
metab_conc_unsup = res_unsup.getConc() / (unsup_mrs.fid_scaling / (unsup_mrs.basis_scaling[0]))
metab_conc_sup = res_sup.getConc() / (supmrs.fid_scaling / (supmrs.basis_scaling[0]))


df = pd.DataFrame()
df.loc[:, 'metabs_unsup'] = pd.Series(metab_conc_unsup, res_unsup.metabs)
df.loc[:, 'metabs_sup'] = pd.Series(metab_conc_sup, res_sup.metabs)


df.loc['H2O', 'metabs_unsup'] = wat_concs
df.loc['H2O', 'metabs_sup'] = wat_concs

df["unsup/sup"] = df["metabs_unsup"] / df["metabs_sup"]


In [ ]:
#Plotting of the fitted data

#Appropriate scale factor
sf3 = supmrs.fid_scaling/unsup_mrs.fid_scaling

x = sup.copy()
a = x.copy()
b = x.copy()

# 2) Create a singleton 5th dimension
a = core.nifti_mrs.reshape(a, (1,), d5='DIM_EDIT')
b = core.nifti_mrs.reshape(b, (1,), d5='DIM_EDIT')

# 3) Merge them along the 5th dimension → size 2
x = core.nifti_mrs.merge([a, b], dimension='DIM_EDIT')

x[:,:,:,:,1] = res_unsup.predictedFID(unsup_mrs, noBaseline=True)
x[:,:,:,:,0] = res_sup.predictedFID(supmrs, noBaseline=True)/sf3

x = proc.align(x, 'DIM_EDIT', ppmlim=(0.2, 4.2))


y = metabs.copy()
y[:] = x[:,:,:,:,1]
z = metabs.copy()
z[:] = x[:,:,:,:,0]

#Optional saving for later plotting
#y.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_fitted/{part}_unsup.nii.gz')
#z.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_fitted/{part}_sup.nii.gz')
#x.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_fitted/{part}_difference.nii.gz')

plt.figure(figsize=(6,4))
y.plot(ppmlim=(0.2,4.2))
z.plot(ppmlim=(0.2,4.2))
x.plot(ppmlim=(0.2,4.2))

ax = plt.gca()

# Recolor lines
lines = ax.get_lines()

diff_line = lines[2]
xdata = diff_line.get_xdata()
ydata = diff_line.get_ydata()

# Apply your modifications:
#   1) multiply by 2
#   2) offset downward by 1e-5
ydata_new = -(2 * ydata + 5)

# Update the line
diff_line.set_ydata(ydata_new)

lines[1].set_color('blue')
lines[0].set_color('red')
lines[2].set_color('orange')

lines[2].set_linewidth(1.8)
lines[1].set_linewidth(1.8)
lines[0].set_linewidth(1.8)

lines[1].set_linestyle((0,(1,1)))

ax.legend(
    [lines[0], lines[1], lines[2]],
    ['GIRF-Corrected Non-WS', 'WS', 'Difference'],
    fontsize=10
)

# Remove y ticks and set label
ax.set_yticks([])
ax.axhline(-5, color='gray', linestyle=':', linewidth=0.7)

#ax.text(ax.get_xlim()[0] - 0.1, 0, '0', va='center', ha='right', fontsize=10, color='gray')
ax.set_ylim(-10, 28)
ax.set_ylabel('Signal', fontsize=11)
ax.set_xlabel("Chemical Shift (ppm)", fontsize=11)
ax.tick_params(axis='x', labelsize=8)
#ax.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.6)
ax.set_title('Fitted Metabolites', fontsize = 14)
#ax.set_title('HLSVD Removal of Residual Water', fontsize = 14)
ax.grid(False)  # turns all grids off first
ax.xaxis.grid(False, which='major')
ax.xaxis.grid(False, which='minor')

plt.tight_layout()
plt.show()


In [ ]:
#Now repeat the process for the diffusion data (after system heating)


raw_unsup_diff = np.load(rf'{NOTEBOOK_DIR}/Data_for_fitting/sLaser/{part}/final_unsup_diffusion.npy')
base_diff= sup.copy()

base_diff[:] = raw_unsup_diff.conj()

base_diff = proc.shift_to_reference(base_diff, 4.65, (4.5, 4.8))
base_diff = proc.phase_correct(base_diff, (4.55, 4.75))

basis = mrs_io.read_basis(rf'{NOTEBOOK_DIR}/basis_sets/slaser_dkd_30.0_h2o')

mrs1_diff = base_diff.mrs(basis=basis)

original_basis_scaling_diff = mrs1_diff.basis_scaling[0]
mrs1_diff.keep = ['H2O']

mrs1_diff.check_Basis(repair=True)
mrs1_diff.rescaleForFitting()


res_water_diff = mrs1_diff.fit(
    ppmlim= (-10,4.55),
    baseline='off',
    model='voigt'
)

wat_concs_diff = res_water_diff.getConc()[0] / (mrs1_diff.fid_scaling / (mrs1_diff.basis_scaling[0]))

plt.figure()
_ = res_water_diff.plot(mrs1_diff)


In [ ]:
#Removal of the water resonance from the unsuppressed data

fid_pred_diff = res_water_diff.predictedFID(mrs1_diff, mode='Full', noBaseline=True)

fid_pred_diff  /= mrs1_diff.scaling['FID']

water_diff = base_diff.copy()
water_diff[:] = fid_pred_diff


metabs_diff = proc.subtract(base_diff, water_diff)
metabs_diff[:] *= 2
metabs_diff = proc.shift_to_reference(metabs_diff, 2.01, (1.9, 2.1), figure=False)
metabs_diff = proc.phase_correct(metabs_diff, (1.9,2.1))


In [ ]:
#Alignment of the before/after heating data 
x = sup.copy()
a = x.copy()
b = x.copy()

# 2) Create a singleton 5th dimension
a = core.nifti_mrs.reshape(a, (1,), d5='DIM_EDIT')
b = core.nifti_mrs.reshape(b, (1,), d5='DIM_EDIT')

# 3) Merge them along the 5th dimension → size 2
x = core.nifti_mrs.merge([a, b], dimension='DIM_EDIT')

x[:,:,:,:,1] = metabs_diff[:]/(wat_concs_diff/wat_concs)
x[:,:,:,:,0] = metabs[:]

x = proc.align(x, 'DIM_EDIT', ppmlim=(0.2, 4.2))


y = metabs.copy()
y[:] = x[:,:,:,:,1]
z = metabs.copy()
z[:] = x[:,:,:,:,0]

#Optional saving
#y.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_raw_diffusion/{part}_unsup_diffusion.nii.gz')
#z.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_raw_diffusion/{part}_unsup.nii.gz')
#x.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_raw_diffusion/{part}_difference.nii.gz')

plt.figure(figsize=(6,4))
z.plot(ppmlim=(0.2,4.2))
y.plot(ppmlim=(0.2,4.2))
x.plot(ppmlim=(0.2,4.2))

ax = plt.gca()

# Recolor lines
lines = ax.get_lines()

diff_line = lines[2]
xdata = diff_line.get_xdata()
ydata = diff_line.get_ydata()

# Apply your modifications:
#   1) multiply by 2
#   2) offset downward by 1e-5
ydata_new = (2 * ydata - 1e-5)

# Update the line
diff_line.set_ydata(ydata_new)

lines[1].set_color('blue')
lines[0].set_color('red')
lines[2].set_color('orange')

lines[2].set_linewidth(1.8)
lines[1].set_linewidth(1.8)
lines[0].set_linewidth(1.8)

lines[1].set_linestyle((0,(1,1)))

ax.legend(
    [lines[0], lines[1], lines[2]],
    ['GIRF-Corrected Non-WS','Diffusion', 'Difference'],
    fontsize=10
)

# Remove y ticks and set label
ax.set_yticks([])
ax.axhline(-1e-5, color='gray', linestyle=':', linewidth=0.7)

#ax.text(ax.get_xlim()[0] - 0.1, 0, '0', va='center', ha='right', fontsize=10, color='gray')
ax.set_ylim(-0.000018, 0.00004)
ax.set_ylabel('Signal', fontsize=11)
ax.set_xlabel("Chemical Shift (ppm)", fontsize=11)
ax.tick_params(axis='x', labelsize=8)
ax.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.6)
ax.set_title('Processed Data for Metabolite Fitting', fontsize = 14)
#ax.set_title('HLSVD Removal of Residual Water', fontsize = 14)
ax.grid(False)  # turns all grids off first
ax.xaxis.grid(False, which='major')
ax.xaxis.grid(False, which='minor')

plt.tight_layout()
plt.show()

In [ ]:
#Fit the post diffusion data 

metabs_diff[:] = x[:,:,:,:,1]

basis_nowater = mrs_io.read_basis(rf'{NOTEBOOK_DIR}/basis_sets/slaser_dkd_30.0_h2o')
basis_nowater.remove_fid_from_basis('H2O')
#basis_nowater.add_fid_to_basis(fid, 'LIP')

unsup_mrs_diff = metabs_diff.mrs(basis=basis_nowater)

unsup_mrs_diff.check_Basis(repair=True)
unsup_mrs_diff.rescaleForFitting(ind_scaling=['Mac'])

unsup_mrs_diff.fid_scaling = supmrs.fid_scaling


res_unsup_diff = unsup_mrs_diff.fit(
    metab_groups=unsup_mrs_diff.parse_metab_groups([ 'Mac']),
    ppmlim=(0.2, 4.2),

    baseline='spline, moderate',
    model='voigt'
)

plotly_fit(unsup_mrs_diff, res_unsup_diff, (0.2, 4.2))

In [ ]:
#Finish constructing data frame for concentration comparisons

metab_conc_unsup = res_unsup.getConc() / (unsup_mrs.fid_scaling / (unsup_mrs.basis_scaling[0]))
metab_conc_unsup_diff = res_unsup_diff.getConc() / (unsup_mrs_diff.fid_scaling / (unsup_mrs_diff.basis_scaling[0]))

df.loc[:, 'metabs_unsup_diff'] = pd.Series(metab_conc_unsup_diff, res_unsup_diff.metabs)


df.loc['H2O', 'metabs_unsup_diff'] = wat_concs

df.loc['tNAA'] = df.loc['NAA'] + df.loc['NAAG']
df.loc['tCho'] = df.loc['GPC'] + df.loc['PCh']
df.loc['Glx']  = df.loc['Glu'] + df.loc['Gln']

if 'Cr+PCr' in df.index:
    df.rename(index={'Cr+PCr': 'tCr'}, inplace=True)

df =df.div(df.loc['H2O',:])
df["unsup/sup"] = df["metabs_unsup"] / df["metabs_sup"]
df["unsup/unsup_diff"] = df["metabs_unsup"] / df["metabs_unsup_diff"]

#Optionally save
#df.to_csv(rf'{NOTEBOOK_DIR}/Data_for_plotting/concetration_ratios/sLaser/{part}.csv', index=True)

In [ ]:
print(df["unsup/sup"])

In [ ]:
print(df["unsup/unsup_diff"])

In [ ]:
#Plotting of the fitted before/after heating data

#appropriate scaling
sf4 = unsup_mrs_diff.fid_scaling/unsup_mrs.fid_scaling



x = sup.copy()
a = x.copy()
b = x.copy()

# 2) Create a singleton 5th dimension
a = core.nifti_mrs.reshape(a, (1,), d5='DIM_EDIT')
b = core.nifti_mrs.reshape(b, (1,), d5='DIM_EDIT')

# 3) Merge them along the 5th dimension → size 2
x = core.nifti_mrs.merge([a, b], dimension='DIM_EDIT')

x[:,:,:,:,1] = res_unsup_diff.predictedFID(unsup_mrs_diff, noBaseline=True)/sf4
x[:,:,:,:,0] = res_unsup.predictedFID(unsup_mrs, noBaseline=True)

x = proc.align(x, 'DIM_EDIT', ppmlim=(0.2, 4.2))


y = metabs.copy()
y[:] = x[:,:,:,:,1]
z = metabs.copy()
z[:] = x[:,:,:,:,0]

#y.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_fitted_diffusion/{part}_unsup_diffusion.nii.gz')
#z.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_fitted_diffusion/{part}_unsup.nii.gz')
#x.save(rf'{NOTEBOOK_DIR}/Data_for_plotting/sLaser_fitted_diffusion/{part}_difference.nii.gz')

plt.figure(figsize=(6,4))

z.plot(ppmlim=(0.2,4.2))
y.plot(ppmlim=(0.2,4.2))
x.plot(ppmlim=(0.2,4.2))

ax = plt.gca()

# Recolor lines
lines = ax.get_lines()

diff_line = lines[2]
xdata = diff_line.get_xdata()
ydata = diff_line.get_ydata()

# Apply your modifications:
#   1) multiply by 2
#   2) offset downward by 1e-5
ydata_new = (2 * ydata - 1)

# Update the line
diff_line.set_ydata(ydata_new)

lines[1].set_color('blue')
lines[0].set_color('red')
lines[2].set_color('orange')

lines[2].set_linewidth(1.8)
lines[1].set_linewidth(1.8)
lines[0].set_linewidth(1.8)

lines[1].set_linestyle((0,(1,1)))

ax.legend(
    [lines[0], lines[1], lines[2]],
    ['before','after', 'Difference'],
    fontsize=10
)

# Remove y ticks and set label
ax.set_yticks([])
ax.axhline(-1, color='gray', linestyle=':', linewidth=0.7)

#ax.text(ax.get_xlim()[0] - 0.1, 0, '0', va='center', ha='right', fontsize=10, color='gray')
ax.set_ylim(-4, 30)
ax.set_ylabel('Signal', fontsize=11)
ax.set_xlabel("Chemical Shift (ppm)", fontsize=11)
ax.tick_params(axis='x', labelsize=8)
#ax.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.6)
ax.set_title('Fitted Data before/after heating', fontsize = 14)
#ax.set_title('HLSVD Removal of Residual Water', fontsize = 14)
ax.grid(False)  # turns all grids off first
ax.xaxis.grid(False, which='major')
ax.xaxis.grid(False, which='minor')

plt.tight_layout()
plt.show()